# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [4]:
import logging
import os
import csv
import joblib, pickle
import json

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

#from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.33.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-project'

experiment=Experiment(ws, experiment_name)

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster = 'automl-cluster'

try:
    comp_trget = ComputeTarget(workspace=ws, name=compute_cluster)
    print('Cluster already exists')
except ComputeTargetException:
    compute_confg = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",max_nodes=4) ## VER
    comp_trget = ComputeTarget.create(ws,compute_cluster,compute_confg)

Cluster already exists


In [9]:
data_link = 'https://raw.githubusercontent.com/fcgomezr/Udacity-Project-3/main/data%20sets/WA_Fn-UseC_-Telco-Customer-Churn.csv'
input_data = TabularDatasetFactory.from_delimited_files(path=data_link, separator = ",")

In [10]:
complete_data = input_data.to_pandas_dataframe()
complete_data 

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,True,False,1,False,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,True,Electronic check,29.85,29.85,False
1,Male,0,False,False,34,True,No,DSL,Yes,No,Yes,No,No,No,One year,False,Mailed check,56.95,1889.50,False
2,Male,0,False,False,2,True,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,True,Mailed check,53.85,108.15,True
3,Male,0,False,False,45,False,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,False,Bank transfer (automatic),42.30,1840.75,False
4,Female,0,False,False,2,True,No,Fiber optic,No,No,No,No,No,No,Month-to-month,True,Electronic check,70.70,151.65,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,True,True,24,True,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,True,Mailed check,84.80,1990.50,False
7039,Female,0,True,True,72,True,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,True,Credit card (automatic),103.20,7362.90,False
7040,Female,0,True,True,11,False,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,True,Electronic check,29.60,346.45,False
7041,Male,1,True,False,4,True,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,True,Mailed check,74.40,306.60,True


In [11]:
# split data into training and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(complete_data,test_size=0.25)
train_data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
5663,Female,0,True,True,37,True,Yes,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month,True,Electronic check,95.25,3314.15,False
4,Female,0,False,False,2,True,No,Fiber optic,No,No,No,No,No,No,Month-to-month,True,Electronic check,70.70,151.65,True
6590,Female,0,False,False,1,True,No,Fiber optic,No,No,No,No,No,No,Month-to-month,True,Electronic check,70.85,70.85,True
5993,Male,0,False,False,2,True,No,Fiber optic,No,Yes,No,No,No,No,Month-to-month,False,Electronic check,74.95,151.75,False
6116,Female,0,False,False,48,True,Yes,DSL,No,Yes,Yes,Yes,Yes,No,One year,True,Bank transfer (automatic),73.85,3581.40,False


In [12]:
train_data.to_csv("traindata.csv")

In [13]:
data_store = ws.get_default_datastore()
data_store.upload(src_dir='./',target_path='./')
ftrain_data = Dataset.Tabular.from_delimited_files(path = [(data_store, './traindata.csv')])

Uploading an estimated of 15 files
Uploading ./.amlignore
Uploaded ./.amlignore, 1 files out of an estimated total of 15
Uploading ./.amlignore.amltmp
Uploaded ./.amlignore.amltmp, 2 files out of an estimated total of 15
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 3 files out of an estimated total of 15
Uploading ./conda_dependencies.yml
Uploaded ./conda_dependencies.yml, 4 files out of an estimated total of 15
Uploading ./hyperparameter_tuning.ipynb
Uploaded ./hyperparameter_tuning.ipynb, 5 files out of an estimated total of 15
Uploading ./model_training.py
Uploaded ./model_training.py, 6 files out of an estimated total of 15
Uploading ./README.md
Uploaded ./README.md, 7 files out of an estimated total of 15
Uploading ./score.py
Uploaded ./score.py, 8 files out of an estimated total of 15
Uploading ./train.py
Uploaded ./train.py, 9 files out of an estimated total of 15
Uploading ./traindata.csv
Uploaded ./traindata.csv, 10 files out of an estimated total of 15
Uploading ./.azure

In [14]:
temp = ftrain_data.drop_columns('Columns1')

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [17]:
# TODO: Put your automl settings here
automl_settings = {    
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             compute_target=comp_trget,
                             task = "classification",
                             training_data=temp,
                             label_column_name="Churn",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             n_cross_validations=7,
                             debug_log = "automl_errors.log",
                             **automl_settings
)

In [18]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)


Submitting remote run.
No run_configuration provided, running on automl-cluster with default configuration
Running on remote compute: automl-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project,AutoML_e92d563d-e00c-42a2-94cb-09f00c0406f4,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business req

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [19]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project,AutoML_e92d563d-e00c-42a2-94cb-09f00c0406f4,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization
DETAILS:      
+---------------------------------+---------------------------------+--------------------------------

{'runId': 'AutoML_e92d563d-e00c-42a2-94cb-09f00c0406f4',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-09-18T18:03:35.57511Z',
 'endTimeUtc': '2021-09-18T18:28:33.150346Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '7',
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-project","subscription_id":"9e65f93e-bdd8-437b-b1e8-0647cd6098f7","resource_group":"aml-quickstarts-158687","workspace_name":"quick-starts-ws-158687","region":"southcentralus","compute_target":"automl-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [20]:
best_automl, best_fit_model = remote_run.get_output()
print(best_fit_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
)), ('lightgbmclassifier', LightGBMClassifier(boosting_type='goss', colsample_bytree=0.2977777777777778, learning_rate=0.015797894736842105, max_bin=250, max_depth=6, min_child_weight=7, min_data_in_leaf=0.013801724137931036, min_split_gain=0.5789473684210527, n_estimators=200, n_jobs=1, num_leaves=71, problem_info=ProblemInfo(), random_state=None, reg_alpha=0.631578947368421, reg_lambda=0.3157894736842105, subsample=1))], verbose=False)), ('28', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(boosting_type='gbdt', colsample_bytree=0.1,

In [21]:
print(best_automl)

Run(Experiment: capstone-project,
Id: AutoML_e92d563d-e00c-42a2-94cb-09f00c0406f4_45,
Type: azureml.scriptrun,
Status: Completed)


In [22]:
#TODO: Save the best model
joblib.dump(best_fit_model, 'best_fit_automl_model.pkl')

['best_fit_automl_model.pkl']

In [18]:
#joblib.dump(best_fit_model,'./outputs/bestmodel.joblib')
#best_class.register_model(model_name='best_fit_automl_model.pkl',model_path='/outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [23]:
from azureml.core.model import Model
model = Model.register(workspace = ws, model_name = 'best_fit_automl_model', model_path = 'best_fit_automl_model.pkl')
print(model.name, model.id, model.version, sep='\t')

Registering model best_fit_automl_model
best_fit_automl_model	best_fit_automl_model:1	1


TODO: In the cell below, send a request to the web service you deployed to test it.

In [24]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, Webservice, AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import azureml.train.automl

# Create the environment
env = best_automl.get_environment()
conda_dep = CondaDependencies()

# Define the packages needed by the model and the script
#conda_dep.add_conda_package("numpy")
#conda_dep.add_conda_package("scikit-learn")
#conda_dep.add_conda_package("pandas")
# Define azureml-defaults as a pip dependency
#conda_dep.add_pip_package("azureml-defaults")
#conda_dep.add_pip_package("azureml-train")

# Adds dependencies to PythonSection of myenv
#env.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script='score.py', environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True)
service = Model.deploy(ws, "customerservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-09-18 18:38:38+00:00 Creating Container Registry if not exists..
2021-09-18 18:48:39+00:00 Registering the environment.
2021-09-18 18:48:39+00:00 Use the existing image.
2021-09-18 18:48:39+00:00 Generating deployment configuration..
2021-09-18 18:48:41+00:00 Submitting deployment to compute.
2021-09-18 18:48:44+00:00 Checking the status of deployment customerservice..
2021-09-18 18:53:20+00:00 Checking the status of inference endpoint customerservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://aa3baa03-9e25-4ed1-88b8-cd7af278b687.southcentralus.azurecontainer.io/score
http://aa3baa03-9e25-4ed1-88b8-cd7af278b687.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
import requests
import json

# URL for the web service
scoring_uri = 'http://aa3baa03-9e25-4ed1-88b8-cd7af278b687.southcentralus.azurecontainer.io/score'

# Set the content type
headers = {'Content-Type': 'application/json'}

# Two sets of data to score, so we get two results back
data = {"data":
        [{
         
            "gender":"Female",
            "SeniorCitizen":0,
            "Partner":"Yes",
            "Dependents":"No",
            "tenure":1,
            "PhoneService":"No",
            "MultipleLines":"No phone service",
            "InternetService":"DSL",
            "OnlineSecurity":"No",
            "OnlineBackup":"Yes",
            "DeviceProtection":"No",
            "TechSupport":"No",
            "StreamingTV":"No",
            "StreamingMovies":"No",
            "Contract":"Month-to-month",
            "PaperlessBilling":"Yes",
            "PaymentMethod":"Electronic check",
            "MonthlyCharges":29.85,
            "TotalCharges":29.85}
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)

print("Response Code : ", resp.status_code)
print("Predicted Value : ",resp.text)

In [25]:
# Web Service Logs
print(service.get_logs())

2021-09-18T18:53:07,813563500+00:00 - iot-server/run 
2021-09-18T18:53:07,807115300+00:00 - rsyslog/run 
2021-09-18T18:53:07,817811500+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-09-18T18:53:07,838277400+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_77e992fdaf2b83b0c81ee1c90ded8d17/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-09-18T18:53:08,181306600+00:00 - iot-server/finish 1 0
2021-09-18T18:53:08,183102400+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (62)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 90
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2021-09-18 18:53:13,906 | root | INFO | Starting up app insights client
logging socket was 

In [26]:
# Delete the service
service.delete()